In [42]:
import json
import glob
import pandas as pd
from sentence_transformers import SentenceTransformer, InputExample, util, SentencesDataset
from sentence_transformers import losses
from sentence_transformers.evaluation import EmbeddingSimilarityEvaluator
from torch.utils.data import DataLoader
from datetime import datetime
import re

In [43]:
def make_connections(d, sub_d):
    file_names = [file.rsplit('\\')[-1] for file in glob.glob(f"text_data/data/{d}_texts/{sub_d}/*.txt")]
    labled_data = {}

    for lable in data:
        labled_data[lable] = []
        for link in data[lable]:
            trim_link, page = link.rsplit('/', 1)

            if page == '':
                page = trim_link.rsplit('/', 1)[1]

            if page+".txt" in file_names:
                labled_data[lable].append(page)
    return labled_data

In [44]:
def write_data(file, labled_data):
    with open('text_data/labled_data/'+file, 'w') as lf:
        json.dump(labled_data, lf, indent=2)

        print("New json file created in text_data->labled_data directory.")

In [45]:
# Load Json data

for file in glob.glob(f"text_data/data/*.json"):
    file = file.split('\\', 1)[1]
    file = file.split('_')
    file1 = file[0]
    file2 = file[2]
    
    file = '_'.join(file)
    print(file1, file2)
    
    with open('text_data/data/'+file, 'r') as fp:
        data = json.load(fp)
        
    data = make_connections(file1, file2)
    
    # Write data to a new file.
    write_data(file, data)

britannica aviation
New json file created in text_data->labled_data directory.
britannica car
New json file created in text_data->labled_data directory.
britannica ped
New json file created in text_data->labled_data directory.
wiki aircraft
New json file created in text_data->labled_data directory.
wiki car
New json file created in text_data->labled_data directory.
wiki ped
New json file created in text_data->labled_data directory.


### Similarity Between Lable and Data

In [ ]:
similarity_df = pd.DataFrame({'Brand': pd.Series(dtype='str'), 'Type': pd.Series(dtype='str'), 'Label': pd.Series(dtype='str'), 'File Name':pd.Series(dtype='str'),
                              'Text': pd.Series(dtype='str'), 'Score': pd.Series(dtype='float')})



# Cosine similarity
def cosine_similarity(labled_data, file_name, brand, typ):
    global similarity_df
    model = SentenceTransformer('all-MiniLM-L6-v2')
    i = 0
    print(len(labled_data))
    for key in labled_data:
        i= i+1
        for value in labled_data[key]:

            with open('text_data/data/'+brand+'_texts/'+typ+'/'+value+'.txt', encoding="utf8") as txt:
                d = txt.read().strip()
                d = d.replace('\n', ' ')
                d = re.sub('\s+', ' ', d)
                d = d.split(' ')
                d = ' '.join(d[:100])
                
            lable_emb = model.encode(key)
            data_emb = model.encode(d)
            cos_sim = util.cos_sim(lable_emb, data_emb)
            
            similarity_df = similarity_df.append({'Brand': brand, 'Type': typ, 'Label': key, 'File Name': value, 'label_type': value+'_'+key, 'Text': d, 'Score': float(cos_sim[0, 0])}, ignore_index=True)
        print(key, i)

    
cosine_similarity(data, file, file1, file2)

In [ ]:
similarity_df['label_type'] = similarity_df['File Name']+'_'+similarity_df['Type']

In [ ]:
# Convert to csv
text_score = similarity_df[['label_type', 'Text', 'Score']]

# text_score.to_csv('text_score.csv')
# similarity_df.to_csv('cosine_sim.csv')
text_score

In [ ]:
# # Prepare data for training.
# brand = 'wiki'
# typ = 'ped'
# with open('text_data/labled_data/'+brand+'_l1_'+typ+'_link.json', 'r') as fp:
#     data = json.load(fp)

# input_example = []
# sum_c = 0
# i=0
# label_values = {}

# for key in data:
#     i= i+1
#     examples = []
#     for value in data[key]:
#         with open('text_data/data/'+brand+'_texts/'+typ+'/'+value+'.txt', encoding="utf8") as txt:
#             d = txt.readlines()
#             d = ' '.join(d)
#             d = d.replace('\n', '')
#             examples.append(d)
#             sum_c+=1
#     label_values[i] = key
#     input_example.append(InputExample(texts=examples[:10], label=float(i/1000)))
#     #if i%10==0:
#     #    break

In [13]:
similarity_df = pd.read_csv('cosine_sim.csv')
similarity_df.dropna(subset=['Text'], how='all', inplace=True)

In [14]:
# Training Data.
text_label = list(zip(list(similarity_df['Text']), list(similarity_df['Score'])))
input_example = []

for key, val in text_label:
    kl = len(key)//2
    texts = [key[:kl], key[kl:]]
    input_example.append(InputExample(texts = texts, label=val/5))

print(len(input_example))
model = SentenceTransformer('distilbert-base-nli-mean-tokens')

# train_dataset = SentencesDataset(input_example, model)

train_dataloader = DataLoader(input_example[:1000], shuffle=True, batch_size=64)
train_loss = losses.CosineSimilarityLoss(model)

25380


In [30]:
#Tune the model
save_path = 'trained_model-'+datetime.now().strftime("%Y-%m-%d_%H-%M-%S")
train_dataset = model.fit(train_objectives=[(train_dataloader, train_loss)], epochs=1, warmup_steps=100, output_path=save_path)
train_dataset

In [31]:
# text_label = list(zip(list(similarity_df['Text']), list(similarity_df['Score'])))
# input_example = list(map(lambda val: InputExample(texts = [val[0]], label=val[1]), text_label))
model

SentenceTransformer(
  (0): Transformer({'max_seq_length': 128, 'do_lower_case': False}) with Transformer model: DistilBertModel 
  (1): Pooling({'word_embedding_dimension': 768, 'pooling_mode_cls_token': False, 'pooling_mode_mean_tokens': True, 'pooling_mode_max_tokens': False, 'pooling_mode_mean_sqrt_len_tokens': False})
)

In [11]:
test_evaluator = EmbeddingSimilarityEvaluator.from_input_examples(input_example, name='sts-test')
final = test_evaluator(model)

In [1]:
final

NameError: name 'final' is not defined

In [33]:
new_model = SentenceTransformer(save_path)

In [34]:
result = new_model.fit(train_objectives=[(train_dataloader, train_loss)], epochs=1, warmup_steps=100)

In [50]:
similarity_df1 = pd.DataFrame({'Brand': pd.Series(dtype='str'), 'Type': pd.Series(dtype='str'), 'Label': pd.Series(dtype='str'), 'File Name':pd.Series(dtype='str'),
                              'Text': pd.Series(dtype='str'), 'Score': pd.Series(dtype='float')})

def cosine_similarity1(labled_data, file_name, brand, typ):
    global similarity_df1
    global new_model
    i = 0
    print(len(labled_data))
    for key in labled_data:
        i= i+1
        for value in labled_data[key]:

            with open('text_data/data/'+brand+'_texts/'+typ+'/'+value+'.txt', encoding="utf8") as txt:
                d = txt.read().strip()
                d = d.replace('\n', ' ')
                d = re.sub('\s+', ' ', d)
                d = d.split(' ')
                d = ' '.join(d[:100])
                
            lable_emb = new_model.encode(key)
            data_emb = new_model.encode(d)
            cos_sim = util.cos_sim(lable_emb, data_emb)
            
            similarity_df1 = similarity_df1.append({'Brand': brand, 'Type': typ, 'Label': key, 'File Name': value, 'label_type': value+'_'+key, 'Text': d, 'Score': float(cos_sim[0, 0])}, ignore_index=True)
        print(key, i)

    
cosine_similarity1(data, file, file1, file2)

307
plodder 1
trafficlane 2
interference 3
walkway 4
crosswalk 5
playmates 6
walker 7
trafficcircle 8
stalker 9
pathway 10
plaintiff 11
circulation 12
footstep 13
nature 14
naturestrip 15
hike 16
bark 17
factors 18
district 19
approach 20
rightofway 21
negligence 22
morning 23
roadsurface 24
persons 25
value 26
tramper 27
reduction 28
totterer 29
tripper 30
viatecture 31
statue 32
transportationsystem 33
wards 34
tunnel 35
route 36
saunter 37
merit 38
footway 39
level 40
eastbound 41
marcher 42
arterial 43
south 44
busdepot 45
majority 46
robinson 47
mall 48
taxistand 49
obstruction 50
bicyclerack 51
pawn 52
prose 53
businessdistrict 54
separation 55
curbless 56
survey 57
violation 58
europe 59
experience 60
adult 61
sense 62
space 63
scale 64
energy 65
tree 66
hour 67
maindrag 68
fashion 69
pedestrianisation 70
exercise 71
pedestrial 72
gateless 73
amount 74
prosaic 75
non 76
walkable 77
crossings 78
pedestriantraffic 79
cobblestone 80
transit 81
force 82
intoxicated 83
account 84
str

In [51]:
similarity_df1.to_csv('new_cosine_sim.csv')

In [52]:
# Convert to csv
text_score = similarity_df1[['label_type', 'Text', 'Score']]
text_score

,label_type,Text,Score
0,List_of_police-related_slang_terms_plodder,Many police-related slang terms exist for poli...,-0.002582
1,Charles_Darwin_plodder,Professional institution: Charles Robert Darwi...,-0.064479
2,Archive_17_plodder,I propose we remove the Notification is to be ...,0.057894
3,Lane_trafficlane,"In road transport, a lane is part of a roadway...",0.414688
4,High-occupancy_vehicle_lane_trafficlane,A high-occupancy vehicle lane (also known as a...,0.353449
...,...,...,...
25390,Speed_limit_vehiculartraffic,"Speed limits on road traffic, as used in most ...",0.169553
25391,Reversible_lane_vehiculartraffic,A reversible lane (British English: tidal flow...,0.124376
25392,Road_signs_in_Italy_vehiculartraffic,Road signs in Italy conform to the general pat...,0.187801
25393,Road_surface_vehiculartraffic,"A road surface (British English), or pavement ...",0.289680
